# Weighted K Means Clustering   


## 1. Introduction

The Weighted K-means clustering is a new k-means clustering algorithm that can automatically weight variables based on the importance of the variables in clustering. Weighted k-means adds a new step to the basic k-means algorithm to update the variable weights based on the current partition of data. It uses a weight calculation formula that minimizes the cost function of clustering given a fixed partition of data.    



 ## 2. The Algorithm

   

I would use the original [paper](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.652.9691&rep=rep1&type=pdf) by Joshua Zhexue Huang, Michael K. Ng, Hongqiang Rong, and Zichen Li as a reference and almost the same set of notations for the variables.   

Let $X= \{ X_{1}, \ X_2,...,\ X_n \}$ be a set of $n$ objects. Object $X_i= \{ x_{i,1}, \ x_{i,2},...,\ x_{i,m}\}$ is characterized by a set of $m$ variables. We want to partition the $n$ objects into $k$ clusters that minimizes the objective function $P$ with unknown variables $U$,$Z$ and $W$ as follows: $$  P(U,Z,W)= \sum_{l=1}^{k}\sum_{i=1}^{n}\sum_{j=1}^{m} u_{i,l} w_j^{\beta} d(x_{i,j}, z_{l,j})  $$ where,  
- $U$ is an $n \times k$ partition matrix, $u_{i,l}$ is a binary variable, and $u_{i,l}=1$ indicates that object $i$ is allocated to cluster $l$;  
- $Z= \{ Z_{1}, \ Z_2,...,\ Z_k \}$ is a set of $k$ vectors representing the centroids of the $k$ clusters;   
- $d(x_{i,j}, z_{l,j}) $ is a distance measure between object $i$ and the centroid of the cluster $l$ on the $j$th variable. We define here $d(x_{i,j}, z_{l,j}) $ as : $$d(x_{i,j}, z_{l,j})= (x_{i,j}-z_{l,j})^{2}.$$   
- $W= \{ w_{1}, \ w_2,...,\ w_m \}$ be the weights for the $m$ variables such that $\sum_{j=1}^{m} w_{j}=1$ , $0 \le w_j \le 1.$   
So to minimize the cost function $P$, we have the Weighted k-means clustering as follows:  

**Weighted k-means Algorithm**    
1. Randomly choose an initial $Z^{0}=\{Z_{1},Z_{2},...,Z_{k}\}$ and randomly generate a set of initial weights $W^{0}=[ w_{1}^{0},\ w_{2}^{0},...,\ w_{m}^{0}] \ (\sum_{j=1}^{m} w_{j}=1)$. Determine $U^{0}$ such that $P(U^{0},Z^{0},W^{0})$ is minimized. Set $t=0$ ;   
2. Let $\hat{Z}=Z^{t}$ and $\hat{W}=W^{t}$, solve problem $P(U,\hat{Z},\hat{W})$ to obtain $U^{t+1}$. If $P(U^{t+1},\hat{Z},\hat{W})=P(U^{t},\hat{Z},\hat{W})$, output $(U^{t},\hat{Z},\hat{W})$ and stop; otherwise go to step 3;   
3. Let $\hat{U}=U^{t+1}$ and $\hat{W}=W^{t}$, solve problem $P(\hat{U},Z,\hat{W})$ to obtain $Z^{t+1}$. If $P(\hat{U},Z^{t+1},\hat{W})=P(\hat{U},Z^{t},\hat{W})$, output $(\hat{U},Z^{t},\hat{W})$ and stop; otherwise go to step 4;   
4. Let $\hat{U}=U^{t+1}$ and $\hat{Z}=Z^{t+1}$, solve problem $P(\hat{U},\hat{Z},W)$ to obtain $W^{t+1}$. If $P(\hat{U},\hat{Z},W^{t+1})=P(\hat{U},\hat{Z},W^{t})$, output $(\hat{U},\hat{Z},W^{t})$ and stop; otherwise, set $t=t+1$ go to step 2.






We start by importing the necessary libraries (pandas and Numpy) and defining a function.

In [1]:
pip install colorama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import math
import random
from colorama import Fore, Back, Style
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
def np_ed(point_1, point_2,A):
    distance = np.sum((np.dot(A,np.transpose(np.square(np.array(point_1) - np.array(point_2))))))
    return distance




## 3. Defining the W k-Means Function

### 3.1 Initializing the first centroids, weights and determining $T$ by minimizing $P$

As the 1st step of the Weighted K Means clustering we initialize the centroid vector and randomly generate the set of initial weights. Then we determine the clusters for each data points in the first iteration.   
**Replacing $U$ by $T$**   
In the original paper, to denote the clusters for each data point in every iteration, the $n \times k$ matrix $U$ was used. The elements $u_{i,l}$ of $U$ is a binary variable, and $u_{i,l}=1$ indicates that object $i$ is allocated to cluster $l$. For brevity and ease, I use a different variable $T$ which is a $(n \times 1)$ column vector, each element of $T$ is a number from the set $\{1,2,...,k\}$ and $t_{i,1}=l$ ,$(l \in \ \{1,2,...,k\})$, means that the $i$th data point belongs to the $l$th cluster. So rewriting our cost function $P$ in terms of $T$, $$  P(T,Z,W)= \sum_{l=1}^{k}\sum_{i=1}^{n}\sum_{j=1}^{m} \mathbb{1}(t_{i,1}=l) w_j^{\beta} d(x_{i,j}, z_{l,j})  $$ where $\mathbb{1}$ is the indicator function defined as 
$$\begin{equation}
\mathbb{1}(t_{i,1}=l) =
    \begin{cases}
        1 & \text{if the $i$th data point belongs to the $l$th cluster} \\
        0 & \text{otherwise}
    \end{cases}
\end{equation}$$




### 3.2 Iteration step for the centroids and weights

Now we implement the W k-means clustering. We carry the three steps until the cost function $P$ becomes constant. In the 2nd step of the W k-means clustering, using the $Z$ and $W$ of the previous iteration we solve for the $T$ that minimizes the cost function $P$. In the following code, after every iteration for the number of data points that belong to each cluster, we get the matrix $T$, the number of data points in each cluster in that iteration and the value of $P$ as output.   
In the 3rd step of the W k-means clustering, using the $T$ and $W$ of the previous iteration we solve for the $Z$ that minimizes the cost function $P$. Solving for $Z$, we get that $P$ is minimized when $$z_{l,j}= \frac{\sum_{i=1}^{n} \mathbb{1}(t_{i,1}=l)\ x_{i,j}}{\sum_{i=1}^{n} \mathbb{1}(t_{i,1}=l)}$$where $1 \le l \le k$ and $0 \le j \le m$.  The following code gives us the new centroids and the value of $P$.   
In the 4th step of the W k-means clustering, using the $T$ and $Z$ of the previous iteration we solve for the $W$ that minimizes the cost function $P$. Solving for $W$, we get that $P$ is minimized when $$w_j= \frac{1}{\sum_{t=0}^{m-1} \left[\frac{D_j}{D_t}\right]^{\frac{1}{\beta - 1}}} \ \ \ \ \ (j\in \{0,1,2,... m-1\} \ \text{and} \ \beta=2)$$ where $$D_t= \sum_{l=1}^{k}\sum_{i=1}^{n} \mathbb{1}(t_{i,1}=l) d(x_{i,t}, z_{l,t})  $$ The following code gives us the new weights and the value of $P$.

In [4]:
def Wkmeans(X, k, max_iter=100, tol=1e-6):
    P=0
    q=1
    A=np.random.choice(X.shape[0], 3, replace=False)
    print(A)
    Z = np.array(X[A])  
    B=2                      #Selecting Beta(B)
    W=np.array([[pow(0.69,B),0,0,0],[0,pow(0.1,B),0,0],[0,0,pow(0.03,B),0], [0,0,0,pow(0.18,B)]])  #Initializing the weights
    for o in range(max_iter):
        T=np.zeros(X.shape[0])
        P1=P
        P=0
        ed=np.zeros((X.shape[0],k))
        for i in range(X.shape[0]):
            for j in range(k):
                ed[i][j] = np_ed(Z[j], X[i],W)
        np.T = np.argmin(ed, axis=1) 

        for i in range(X.shape[0]):
            P =  P + np_ed(X[i], Z[np.T[i]],W)
    
        print(Fore.LIGHTBLUE_EX,"The new clusters for the",q,"th iteration")    
        printmd("**The Matrix T :**")
        print(np.T)
        printmd("**No. of data points in 1st cluster:**")            #The new clusters in this iteration
        print(np.count_nonzero(np.T == 0))
        printmd("**No. of data points in 2nd cluster:**") 
        print(np.count_nonzero(np.T == 1))
        printmd("**No. of data points in 3rd cluster:**")
        print(np.count_nonzero(np.T == 2))  
        printmd("**Value of P:**")
        print(P,"\n")
        if (P1==P): 
            break
        
        P1=P
        P=0
        for j in range(k):
            Z[j]= np.sum(X[np.T==j],axis=0)/np.count_nonzero(np.T == j)

        for i in range(X.shape[0]):
             P=  P + np_ed(X[i], Z[np.T[i]],W)
           
        print(Fore.LIGHTBLUE_EX,"The new centroids for the",q,"th iteration")  
        printmd("**Centroids:**")
        print(Z)                                                     #The new clusters in this iteration
        printmd("**Value of P:**")
        print(P,"\n")
        if (P1==P): 
            break
    
        P1= P
        P=0
        D=np.zeros(X.shape[1])
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                D[j] = D[j] + np.square(X[i][j]-Z[np.T[i]][j])
        D1=np.zeros(X.shape[1])
        for j in range(X.shape[1]):
            D1[j] = 1/(pow((D[j]),1/(B-1)))
        
        W=np.zeros((X.shape[1],X.shape[1]))
        for i in range(X.shape[1]):
            for j in range(X.shape[1]):
                if (i==j):
                    W[i][j]= pow(1/((pow((D[i]),1/(B-1)))*(np.sum(D1))),B)
                else:
                    W[i][j]= 0
           
        for i in range(X.shape[0]):
            P= P + np_ed(X[i], Z[np.T[i]],W)
        print(Fore.LIGHTBLUE_EX ,"The new weights for the",q,"th iteration")
        printmd("**The Weight Matrix:**")
        print(W)                                                               #The new weights in this iteration
        printmd("**Value of P:**")
        print(P,"\n") 
        if (P1==P): 
            break
        q=q+1
    print(Fore.LIGHTMAGENTA_EX,"The number of iterations:",q)

## 4. Implementing the W K-means clustering for IRIS dataset

Here I implement the Weighted K means clustering algorithm on the [IRIS dataset](https://archive.ics.uci.edu/ml/datasets/iris). The data set contains 3 classes of 50 instances each, where each class refers to a type of iris plant namely **Iris-setosa, Iris-versicolor** and **Iris-virginica**.

For the IRIS dataset, we have the no. of data points $n=150$, the number of clusters $k=3$ and the no. of variables $m=4$. So our cost function $P$ for the IRIS dataset is  $$  P(T,Z,W)= \sum_{l=1}^{3}\sum_{i=1}^{150}\sum_{j=0}^{3} \mathbb{1}(t_{i,1}=l) w_j^{\beta} d(x_{i,j}, z_{l,j})  $$

In [5]:
iris=pd.read_csv ("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
                   names = ["Sepal Length", "Sepal Width", "Petal Length", "Petal Width", "Class" ])  #Reading the csv file
X1 = iris.iloc[:,0:4]
y = iris.iloc[:,-1]
X=np.array(X1)
Wkmeans(X,3)

[122 148 135]
 The new clusters for the 1 th iteration


**The Matrix T :**

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 2 1
 1 1 1 1 1 1 2 2 1 1 1 0 1 1 0 1 1 1 0 0 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]


**No. of data points in 1st cluster:**

8


...

50


**No. of data points in 3rd cluster:**

47


**Value of P:**

3.095508867152213 

 The number of iterations: 11
